<a href="https://colab.research.google.com/github/PedroFatec/Processamento-de-Linguagem-Natural-A914-N-D.S.M.-113-20242/blob/main/Aula8/Aula8_22_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 08 - Introdução a ML para PLN
Aluno: Pedro Henrique Figueira


### Fundamento de Machine Learning -> Aplicação Matemática para Fenômenos observáveis





O intuito destes códigos são realizar uma análise de sentimentos em resenhas de filmes usando os modelos Naive Bayes Multinomial e SVM. Ele utiliza o dataset movie_reviews da biblioteca NLTK, que contém textos rotulados como positivos ou negativos.

Os textos são processados e transformados em vetores numéricos com TF-IDF, enquanto os rótulos são convertidos para valores numéricos. Após dividir os dados em treino e teste, os dois modelos são treinados no conjunto de treino.

Por fim, os modelos são avaliados no conjunto de teste, e métricas como precisão e F1-score são exibidas para medir o desempenho de cada um na classificação de sentimentos

In [ ]:
# Passo 1 - Criar o Corpus
corpus = [
    ("Eu amo PLN", "positivo"),  # Exemplo rotulado como positivo
    ("Eu odeio bugs", "negativo"),  # Exemplo rotulado como negativo
    ("Amo resolver problemas", "positivo"),  # Outro exemplo positivo
    ("Odeio erros", "negativo")  # Outro exemplo negativo
]
# O corpus contém textos de exemplo e suas respectivas classes (positivo ou negativo).

# Passo 2 - Processar o Texto
import re
from collections import defaultdict, Counter

# Função para preprocessar o texto: remove maiúsculas e divide o texto em palavras
def preprocess_text(text):
    return re.findall(r'\b\w+\b', text.lower())  # Encontra palavras (sequências de caracteres alfanuméricos)

# Processa cada texto do corpus para obter as palavras e mantém a associação com o rótulo
processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)
# Exemplo de saída: [(['eu', 'amo', 'pln'], 'positivo'), ...]

# Passo 3 - Calculando probabilidades
# Contador para armazenar o número de exemplos por classe
class_counts = Counter()
# Contador para armazenar o número de ocorrências de cada palavra por classe
word_counts = defaultdict(Counter)
# Dicionário para armazenar o total de palavras por classe
total_words = defaultdict(int)

# Itera sobre o corpus processado para calcular as contagens
for words, label in processed_corpus:
    class_counts[label] += 1  # Incrementa o contador da classe
    for word in words:  # Para cada palavra no texto
        word_counts[label][word] += 1  # Incrementa a contagem da palavra para a classe
        total_words[label] += 1  # Incrementa o total de palavras para a classe

# Calcula o total de exemplos no corpus
total_examples = sum(class_counts.values())
# Calcula as probabilidades a priori de cada classe
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}

# Função para calcular a probabilidade condicional com suavização de Laplace
def conditional_probability(word, label, alpha=1):
    return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo Texto
# Função para prever a classe de um texto
def predict(text):
    words = preprocess_text(text)  # Preprocessa o texto
    probabilities = {}  # Dicionário para armazenar as probabilidades de cada classe

    for label in class_counts.keys():
        probabilities[label] = prior_probabilities[label]  # Inicializa com a probabilidade a priori da classe
        for word in words:  # Para cada palavra no texto
            probabilities[label] *= conditional_probability(word, label)  # Multiplica pela probabilidade condicional
    # Retorna a classe com a maior probabilidade e as probabilidades calculadas
    return max(probabilities, key=probabilities.get), probabilities

# Passo 5 - Teste com um novo texto
novo_texto = "Eu amo resolver bugs"
classe, probs = predict(novo_texto)  # Prediz a classe do novo texto

# Exibe os resultados
print(f"Texto: '{novo_texto}'")
print(f"Classe prevista: '{classe}'")
print("Probabilidades:")
for label, prob in probs.items():
    print(f"  {label}: {prob}")


[(['eu', 'amo', 'pln'], 'positivo'), (['eu', 'odeio', 'bugs'], 'negativo'), (['amo', 'resolver', 'problemas'], 'positivo'), (['odeio', 'erros'], 'negativo')]
Texto: 'Eu amo resolver bugs'
Classe prevista: 'positivo'
Probabilidades:
  positivo: 0.00040980807321904243
  negativo: 0.0003048315805517451


In [ ]:
# Passo 1: Importação das bibliotecas
from sklearn.feature_extraction.text import TfidfVectorizer  # Para transformar textos em vetores numéricos usando TF-IDF
from sklearn.svm import SVC  # Para criar o modelo de classificação usando uma Máquina de Vetores de Suporte (SVM)
from sklearn.model_selection import train_test_split  # Para dividir os dados em conjuntos de treino e teste
from sklearn.metrics import classification_report  # Para avaliar a performance do modelo

# Passo 2: Dados Exemplo
corpus = [
    "Eu amo PLN",  # Texto 1
    "Eu odeio bugs",  # Texto 2
    "Eu amo resolver problemas",  # Texto 3
    "Odeio erros",  # Texto 4
    "Amo programação",  # Texto 5
    "Não gosto de falhas"  # Texto 6
]
classes = ["negativo", "negativo", "positivo", "negativo", "positivo", "negativo"]
# Cada texto do corpus é associado a uma classe ('positivo' ou 'negativo').

# Passo 3: Pré-processamento e vetorização
# O TfidfVectorizer transforma os textos do corpus em uma matriz de características numéricas.
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)  # Transforma os textos em vetores numéricos baseados em TF-IDF
y = classes  # Alvo das classes

# Passo 4: Dividir os dados e Treinar o modelo
# Divide os dados em conjuntos de treino e teste.
# 70% para treino e 30% para teste. O parâmetro random_state garante reprodutibilidade.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinamento do modelo SVM com kernel linear.
svm_model = SVC(kernel='linear')  # Define o modelo de SVM com um kernel linear
svm_model.fit(X_train, y_train)  # Treina o modelo com os dados de treino

# Passo 5: Avaliar o modelo
y_pred = svm_model.predict(X_test)  # Realiza previsões no conjunto de teste
print(classification_report(y_test, y_pred))  # Exibe o relatório de classificação


              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         2
    positivo       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Passo 1: Importar Bibliotecas
import nltk  # Biblioteca para processamento de linguagem natural
from sklearn.feature_extraction.text import TfidfVectorizer  # Para transformar texto em vetores numéricos (TF-IDF)
from sklearn.model_selection import train_test_split  # Para dividir os dados em treino e teste
from sklearn.naive_bayes import MultinomialNB  # Classificador Naive Bayes Multinomial
from sklearn.svm import SVC  # Classificador baseado em Máquinas de Vetores de Suporte (SVM)
from sklearn.metrics import classification_report  # Para avaliar a performance do modelo
from sklearn.preprocessing import LabelEncoder  # Para codificar rótulos em valores numéricos

# Baixar o dataset de exemplo
nltk.download('movie_reviews')  # Faz o download do dataset "movie_reviews" da biblioteca NLTK
from nltk.corpus import movie_reviews  # Importa o dataset de resenhas de filmes

# Passo 2: Preparação dos dados
# Coleta de textos e classes do dataset
documents = [(" ".join(movie_reviews.words(fileid)), category)  # Junta todas as palavras de um texto
             for category in movie_reviews.categories()  # Para cada categoria (positivo/negativo)
             for fileid in movie_reviews.fileids(category)]  # Para cada arquivo associado à categoria

# Separar textos e rótulos em listas distintas
texts, labels = zip(*documents)

# Transformar rótulos (positivo/negativo) em valores numéricos (0 e 1)
label_encoder = LabelEncoder()  # Instancia o codificador de rótulos
labels = label_encoder.fit_transform(labels)  # Codifica os rótulos como números

# Dividir os dados em conjuntos de treino (70%) e teste (30%)
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels, test_size=0.3, random_state=42
)

# Passo 3: Representação do texto com TF-IDF
# Criar o vetorizador TF-IDF com limite de 5.000 palavras mais frequentes
vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar e transformar os textos de treino em vetores numéricos
X_train = vectorizer.fit_transform(texts_train)
# Transformar os textos de teste usando o mesmo vetorizador
X_test = vectorizer.transform(texts_test)

# Passo 4: Treinar os modelos
# Treinar o modelo Naive Bayes Multinomial
nb_model = MultinomialNB()  # Instancia o modelo
nb_model.fit(X_train, labels_train)  # Treina o modelo com os dados de treino

# Fazer predições com o modelo Naive Bayes
nb_predictions = nb_model.predict(X_test)

# Treinar o modelo SVM com kernel linear
svm_model = SVC(kernel='linear')  # Instancia o modelo SVM com kernel linear
svm_model.fit(X_train, labels_train)  # Treina o modelo com os dados de treino

# Fazer predições com o modelo SVM
svm_predictions = svm_model.predict(X_test)

# Passo 5: Avaliação dos modelos
# Avaliação do Naive Bayes
print("Naive Bayes Performance:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))

# Avaliação do SVM
print("SVM Performance:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

